# 1.0 Introduction to conversational software

In [1]:
import time

def respond(message):
    print('BOT: I can hear you! You said {}'.format(message))

def send_message(message):
    print('USER: {}'.format(message))
    time.sleep(2)
    return respond(message)

In [2]:
send_message('Hello!')

USER: Hello!
BOT: I can hear you! You said Hello!


# 1.1 Creating a personality

In [3]:
responses = {
    "What's your name?": "My name is Echobot",
    "What's the weather today?": "It's sunny!"
}

def respond(message):
    if message in responses:
        return responses[message]

In [4]:
respond("What's your name?")

'My name is Echobot'

In [5]:
responses = {
    "What's today's weather?": "It's {} today"
}

weatherToday = 'cloudy'

def respond(message):
    if message in responses:
        return responses[message].format(weatherToday)

In [6]:
respond("What's today's weather?")

"It's cloudy today"

In [7]:
responses = {
    "What's your name?": [
        "My name echobot",
        "They call me echobot",
        "The name's bot, echobot"
    ]
}

import random

def respond(message):
    if message in responses:
        return random.choice(responses[message])

In [8]:
respond("What's your name?")

'They call me echobot'

In [9]:
responses = ['Tell me more!', "Why do you think that?"]

import random

def respond(message):
    return random.choice(responses)

In [10]:
respond("I think you're really great")

'Tell me more!'

# 1.2 Text processing with regex

In [11]:
import re

pattern = "Do you remember .*"
message = "Do you remember when you ate strawberries in the garden"
match = re.search(pattern, message)

if match:
    print("String matches!")

String matches!


In [12]:
pattern = "if (.*)"
message = "What would happen if bots took over the world!"
match = re.search(pattern, message)
match.group(0)

'if bots took over the world!'

In [13]:
match.group(1)

'bots took over the world!'

In [14]:
import re

def swap_pronouns(phrase):
    if 'I' in phrase:
        return re.sub('I', 'you', phrase)
    if 'my' in phrase:
        return re.sub('my', 'your', phrase)
    else:
        return phrase

In [15]:
swap_pronouns('I walk my dog')

'you walk my dog'

In [16]:
pattern = "do you remember (.*)"
message = "do you remember when you ate strawberries in the garden"
phrase = re.search(pattern, message)
phrase.group(0)

'do you remember when you ate strawberries in the garden'

In [17]:
phrase.group(1)

'when you ate strawberries in the garden'

In [18]:
phrase = swap_pronouns(phrase.group(1))
phrase

'when you ate strawberries in the garden'

# 2.0 Understanding intents and entities

- Intents
- Entities: NER (Named entity recognition)

## Regular expression to recognize intents and exercises
- Simpler than machine learning approaches
- Highly computationally efficient
- Debugging regex is difficult

In [19]:
re.search(r"(hello|hey|hi)", "hey there!") is not None

True

In [20]:
re.search(r"(hello|hey|hi)", "which one?") is not None

True

In [21]:
re.search(r"\b(hello|hey|hi)\b", "hey there!") is not None

True

In [22]:
re.search(r"\b(hello|hey|hi)\b", "which one?") is not None

False

In [23]:
# Entity recognition
pattern = re.compile('[A-Z]{1}[a-z]*')

message = """
Mary is a friend of mine,
she studied at Oxford and
now works at Google
"""

In [24]:
pattern.findall(message)

['Mary', 'Oxford', 'Google']

# 2.1 Word vectors

- Try to represent meaning of words
- Words with similar context have similar vector
- GloVe algorithm (cousin of word2vec)
- spaCy

In [25]:
import spacy

In [26]:
nlp = spacy.load('en')

nlp.vocab.vectors_length

0

In [27]:
doc = nlp('hello can you help me?')

for token in doc:
    print("{}: {}".format(token, token.vector[:3]))

hello: [ 4.3282194 -3.6430755 -2.1657367]
can: [2.9265766  0.58220786 1.1047966 ]
you: [-2.1102378  2.1661057  2.5413637]
help: [ 3.9393585  1.7003576 -0.3030237]
me: [ 2.534314   -0.22819562  2.4051433 ]
?: [ 1.803947  -1.7313943  2.2867446]


## Similarity

- Direction of vectors matters
- "Distance" between words = angle between the vectors
- Cosine similarity:
    - 1: if vectors point in the same direction
    - 0: if they are perpendicular
    - -1: if they point in opposite direction

In [28]:
doc = nlp("cat")
doc.similarity(nlp("can"))

/home/anonymous/anaconda3/lib/python3.6/runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  "__main__", mod_spec)


0.3167076852929806

In [29]:
doc.similarity(nlp("dog"))

/home/anonymous/anaconda3/lib/python3.6/runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  "__main__", mod_spec)


0.7952184229586672

# 2.2 Intents and classification

- A classifier predicts the intent label from a sentence
- Use training data to tune classifer
- Use testing data to evaluate performance
- Accuracy: fraction of correctly predicted labels

In [30]:
from collections import defaultdict
import gzip
import numpy as np
import os
import pandas as pd
import pickle

In [31]:
os.getcwd()

'/home/anonymous/Documents/github/machine-learning/building-chatbots-in-python'

In [32]:
os.listdir()

['building-chatbots-in-python.ipynb',
 '.ipynb_checkpoints',
 'atis-fold.ipynb',
 'data',
 'atis-dataset.ipynb']

In [33]:
os.uname()

posix.uname_result(sysname='Linux', nodename='FOIAI', release='4.15.0-47-generic', version='#50-Ubuntu SMP Wed Mar 13 10:44:52 UTC 2019', machine='x86_64')

In [34]:
sorted(os.listdir('data'))

['atis',
 'atis-dataset',
 'atis-dataset.zip',
 'atis.fold0.pkl.gz',
 'atis.fold1.pkl.gz',
 'atis.fold2.pkl.gz',
 'atis.fold3.pkl.gz',
 'atis.fold4.pkl.gz',
 'atis.pkl.gz',
 'atis.zip',
 'hotels.csv',
 'hotels.db']

In [35]:
names = ['label', 'sentence']

intentsDF = pd.read_csv('data/atis/atis_intents.csv', names=names)
trainDF = pd.read_csv('data/atis/atis_intents_train.csv', names=names)
testDF = pd.read_csv('data/atis/atis_intents_test.csv', names=names)

In [36]:
intentsDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4978 entries, 0 to 4977
Data columns (total 2 columns):
label       4978 non-null object
sentence    4978 non-null object
dtypes: object(2)
memory usage: 77.9+ KB


In [37]:
intentsDF.head()

,label,sentence
0,atis_flight,i want to fly from boston at 838 am and arriv...
1,atis_flight,what flights are available from pittsburgh to...
2,atis_flight_time,what is the arrival time in san francisco for...
3,atis_airfare,cheapest airfare from tacoma to orlando
4,atis_airfare,round trip fares from pittsburgh to philadelp...


In [38]:
trainDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4834 entries, 0 to 4833
Data columns (total 2 columns):
label       4834 non-null object
sentence    4834 non-null object
dtypes: object(2)
memory usage: 75.6+ KB


In [39]:
trainDF.head()

,label,sentence
0,atis_flight,i want to fly from boston at 838 am and arriv...
1,atis_flight,what flights are available from pittsburgh to...
2,atis_flight_time,what is the arrival time in san francisco for...
3,atis_airfare,cheapest airfare from tacoma to orlando
4,atis_airfare,round trip fares from pittsburgh to philadelp...


In [40]:
testDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 2 columns):
label       800 non-null object
sentence    800 non-null object
dtypes: object(2)
memory usage: 12.6+ KB


In [41]:
testDF.head()

,label,sentence
0,atis_flight,i would like to find a flight from charlotte ...
1,atis_airfare,on april first i need a ticket from tacoma to...
2,atis_flight,on april first i need a flight going from pho...
3,atis_flight,i would like a flight traveling one way from ...
4,atis_flight,i would like a flight from orlando to salt la...


In [42]:
trainSentences = trainDF['sentence'].values.tolist()
trainSentences[:2]

[' i want to fly from boston at 838 am and arrive in denver at 1110 in the morning',
 ' what flights are available from pittsburgh to baltimore on thursday morning']

In [43]:
trainLabels = trainDF['label'].values.tolist()
trainLabels[:2]

['atis_flight', 'atis_flight']

In [44]:
X_train_shape = (len(trainSentences), nlp.vocab.vectors_length)
X_train_shape

(4834, 0)

In [45]:
X_train = np.zeros(X_train_shape)
X_train

array([], shape=(4834, 0), dtype=float64)

In [46]:
test = np.zeros((5000,4))
test

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

```python
for sentence in trainSentences:
    X_train[1, :] = nlp(sentence).vector
```

## Nearest neighbor classification

Simplest solution:
- Look for the labeled example that's most similar
- Use its intent as a best guess

In [47]:
from sklearn.metrics.pairwise import cosine_similarity

In [48]:
sampleTestMessage = testDF.iloc[0, 1]
sampleTestMessage

' i would like to find a flight from charlotte to las vegas that makes a stop in st. louis'

In [49]:
test_x = nlp(sampleTestMessage).vector
np.shape(test_x)

(96,)

```python
scores = [cosine_similarity(X[i,:], test_x) for i in range(len(trainSentences))]

trainLabels[np.argmax(scores)]
```

## Support vector machines

SVM / SVC

```python
from sklearn.svm import SVC
clf = SVC()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
```

# 2.3 Entity extraction

- Keywords don't work for entities you haven't seen before
- Use contextual clues:
    - Spelling
    - Capitalization
    - Words occurring before and after
- Pattern recognition

In [50]:
doc = nlp("my friend Mary has worked at Google since 2009")
doc

my friend Mary has worked at Google since 2009

In [51]:
for ent in doc.ents:
    print(ent.text, ent.label_)

Mary PERSON
Google ORG
2009 DATE


## Roles

In [52]:
import re

In [53]:
pattern1 = re.compile('.* from (.*) to (.*)')
string1 = "I want a flight from Tel Aviv to Bucharest"
pattern1.match(string1)

<_sre.SRE_Match object; span=(0, 42), match='I want a flight from Tel Aviv to Bucharest'>

In [54]:
pattern2 = re.compile('.* to (.*) from (.*)')
string2 = 'show me flights to Shanghai from Singapore'
pattern2.match(string2)

<_sre.SRE_Match object; span=(0, 42), match='show me flights to Shanghai from Singapore'>

## Dependency parsing

In [55]:
doc = nlp('a flight to Shanghai from Singapore')
shanghai, singapore = doc[3], doc[5]

In [56]:
list(shanghai.ancestors)

[to, flight]

In [57]:
list(singapore.ancestors)

[from, flight]

In [58]:
doc = nlp("let's see that jacket in red and some blue jeans")

for token in doc:
    print(token.text, token.pos_, token.dep_)

let VERB ROOT
's PRON nsubj
see VERB ccomp
that DET det
jacket NOUN dobj
in ADP prep
red NOUN pobj
and CCONJ cc
some DET det
blue ADJ amod
jeans NOUN conj


In [59]:
items = [doc[4], doc[10]]
items

[jacket, jeans]

In [60]:
colours = [doc[6], doc[9]]
colours

[red, blue]

In [61]:
for color in colours:
    print('The current color: {0}'.format(color))
    for a in color.ancestors:
        print(a)

The current color: red
in
jacket
see
let
The current color: blue
jeans
jacket
see
let


In [62]:
for color in colours:
    for tok in color.ancestors:
        if tok in items:
            print("color {} belongs to item {}".format(color, tok))
            #Without this break, there will be 1 more statement
            break

color red belongs to item jacket
color blue belongs to item jeans


# 2.4 Robust NLU with Rasa

- Library for intent recognition and entity extraction
- Based on spaCy, scikit-learn and other libraries
- Built-in support for chatbot specific tasks

In [63]:
from rasa_nlu.training_data import load_data

In [64]:
trainDF.iloc[21:23,:]

,label,sentence
21,atis_flight,please list all first class flights on united...
22,atis_aircraft,what kinds of planes are used by american air...


In [65]:
intentsDF.iloc[21:23,:]

,label,sentence
21,atis_flight,show me the flights from san diego to newark
22,atis_flight,please list all first class flights on united...


In [66]:
testDF.iloc[21:23,:]

,label,sentence
21,atis_flight,show flights wednesday evening from st. louis...
22,atis_flight,which flights travel from kansas city to los ...


# 3. Virtual assistants and accessing data

In [67]:
import sqlite3

conn = sqlite3.connect('data/hotels.db')
c = conn.cursor()

In [68]:
c.execute('SELECT name FROM sqlite_master WHERE type="table"')
c.fetchall()

[('hotels',)]

In [69]:
c.execute('SELECT * FROM hotels LIMIT 5')
c.fetchall()

[('Hotel for Dogs', 'mid', 'east', 3),
 ('Hotel California', 'mid', 'north', 3),
 ('Grand Hotel', 'hi', 'south', 5),
 ('Cozy Cottage', 'lo', 'south', 2),
 ("Ben's BnB", 'hi', 'north', 4)]

In [70]:
c.execute('SELECT * FROM hotels WHERE area="south" AND price="hi"')
c.fetchall()

[('Grand Hotel', 'hi', 'south', 5)]

#### db => pandas

In [71]:
import pandas as pd

In [72]:
def sqlite3_to_csv():
    conn = sqlite3.connect('data/hotels.db')
    c = conn.cursor()
    c.execute('SELECT name FROM sqlite_master WHERE type="table"')
    tables = c.fetchall()
    for tup in tables:
        tableName = tup[0]
        table = pd.read_sql_query("SELECT * FROM %s" % tableName, conn)
        filepath = 'data/' + tableName + '.csv'
        table.to_csv(filepath, index_label='index')
    conn.close()

In [73]:
%%time
sqlite3_to_csv()

CPU times: user 128 ms, sys: 0 ns, total: 128 ms
Wall time: 126 ms


In [74]:
df = pd.read_csv('data/hotels.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 5 columns):
index    7 non-null int64
name     7 non-null object
price    7 non-null object
area     7 non-null object
stars    7 non-null int64
dtypes: int64(2), object(3)
memory usage: 360.0+ bytes


In [75]:
df

,index,name,price,area,stars
0,0,Hotel for Dogs,mid,east,3
1,1,Hotel California,mid,north,3
2,2,Grand Hotel,hi,south,5
3,3,Cozy Cottage,lo,south,2
4,4,Ben's BnB,hi,north,4
5,5,The Grand,hi,west,5
6,6,Central Rooms,mid,center,3


# 3.1. Exploring a DB with natural language

# 3.2. Incremental slot filling and negation

In [76]:
doc = nlp('not sushi, maybe pizza?')
indices = [1,4]
ents, negated_ents = [], []
start = 0

In [77]:
for i in indices:
    phrase = '{}'.format(doc[start:i])
    if 'not' in phrase or "n't" in phrase:
        negated_ents.append(doc[i])
    else:
        ents.append(doc[i])
    start = i

In [78]:
negated_ents

[sushi]

In [79]:
ents

[pizza]

# 4. Stateful bots